In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys

In [3]:
project_path = '/content/drive/My Drive/ML project 2/'
sys.path.append(project_path)

In [4]:
from levin.SegFormer.segformer import SegFormer
from levin.road_data import RoadSegmentationDataset
from torch.utils.data import DataLoader
import torchvision.transforms as T
import os
import torch

In [6]:
# Construct the absolute path to the data folder
image_dir = os.path.join(project_path, "data", "training", "augmented", "images")
mask_dir = os.path.join(project_path, "data", "training", "augmented", "masks")

print(f"Image directory: {image_dir}")
print(f"Mask directory: {mask_dir}")

# Check if the paths exist
if not os.path.exists(image_dir):
    raise FileNotFoundError(f"Image directory not found: {image_dir}")
if not os.path.exists(mask_dir):
    raise FileNotFoundError(f"Mask directory not found: {mask_dir}")

# Define transforms for images and masks
image_transform = T.Compose([
    T.Resize((512, 512)),  # Resize images to 512x512
    T.ToTensor(),          # Convert to PyTorch tensor
    T.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize to [-1, 1]
])

mask_transform = T.Compose([
    T.Resize((512, 512)),  # Resize masks to 512x512
    T.ToTensor()           # Convert to PyTorch tensor (binary mask stays as float)
])

image_filenames = sorted(os.listdir(image_dir))
mask_filenames = sorted(os.listdir(mask_dir))

# Split filenames into train and test sets
#train_images, test_images, train_masks, test_masks = train_test_split(
#    image_filenames, mask_filenames, test_size=0.2, random_state=42
#)

# Define datasets for train and test sets
train_dataset = RoadSegmentationDataset(
    image_dir=image_dir,
    mask_dir=mask_dir,
    transform=image_transform,
    target_transform=mask_transform
)

# test_dataset = RoadSegmentationDataset(
#     image_dir=image_dir,
#     mask_dir=mask_dir,
#     transform=image_transform,
#     target_transform=mask_transform
# )

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=0)

criterion = torch.nn.BCEWithLogitsLoss()
save_path = os.path.join(project_path, "levin", "trained_models", "segformer_raw.pt")
model = SegFormer()
model.train(train_loader, criterion, epochs=10, save_path=save_path)

Image directory: /content/drive/My Drive/ML project 2/data/training/augmented/images
Mask directory: /content/drive/My Drive/ML project 2/data/training/augmented/masks


Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/segformer-b0-finetuned-ade-512-512 and are newly initialized because the shapes did not match:
- decode_head.classifier.bias: found shape torch.Size([150]) in the checkpoint and torch.Size([1]) in the model instantiated
- decode_head.classifier.weight: found shape torch.Size([150, 256, 1, 1]) in the checkpoint and torch.Size([1, 256, 1, 1]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10, Loss: 0.5021
Epoch 2/10, Loss: 0.2935
Epoch 3/10, Loss: 0.2223
Epoch 4/10, Loss: 0.1881
Epoch 5/10, Loss: 0.1672
Epoch 6/10, Loss: 0.1519
Epoch 7/10, Loss: 0.1414
Epoch 8/10, Loss: 0.1345
Epoch 9/10, Loss: 0.1258
Epoch 10/10, Loss: 0.1202
Model saved to /content/drive/My Drive/ML project 2/levin/trained_models/segformer_raw.pt
